In [14]:
from langchain import hub

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.utilities.tavily_search import TavilySearchAPIWrapper

from langchain.tools import BraveSearch, BaseTool, StructuredTool, tool

from langchain_openai import ChatOpenAI

from langchain.agents import AgentExecutor
from langchain.agents import create_openai_tools_agent

from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, PromptTemplate, chat
from langchain.schema import AIMessage, HumanMessage, SystemMessage

from langchain.pydantic_v1 import BaseModel, Field

from typing import Optional, Type

## Build Web Searching Tool (Tavily) for Agent

In [9]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

In [ ]:
tavily_tool.invoke("Baltimore Gas and Electric climate-aware planning")

## Build Backup Search Tool

In [17]:
brave_api='BSAmw298lpAtPuiP5crDrAtOpGPgf5U'

In [18]:
brave_tool = BraveSearch.from_api_key(api_key=brave_api, search_kwargs={"count": 4}, description='a search engine. useful to use when other tools do not return results.')

In [ ]:
brave_tool.invoke("Baltimore Gas and Electric climate-aware planning")

In [20]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")


class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to answer questions using the internet."
    args_schema: Type[BaseModel] = SearchInput

    def __init__(self, tavily, brave): 
        self.tavily = tavily
        self.brave = brave

    def _run(
        self, query: str) -> str:
        """Use the tool."""
        output = self.tavily.run(query)
        if not output:
            output = self.brave.run(query)
        
        return output

    async def _arun(
        self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")


In [21]:
custom_search = CustomSearchTool(tavily_tool, brave_tool)

ValueError: "CustomSearchTool" object has no field "tavily"

## Get CRMM Text file 

In [5]:
with open('Climate Resilience Maturity Model.txt', "r", encoding='utf-8') as f:
    crmm_raw = f.read()
# crmm_raw

## Build Chat Prompt for Agent

In [6]:
chat_prompt = ChatPromptTemplate.from_messages([SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['crmm_info'], 
                    template="You are an analyst who uses the Climate Risk Maturity Model to summarize how infrastructure providers" 
                     " are preparing for climate change. You are critical of providers, and only give them credit for progress in a capability if specific and direct evidence is provided." 
                     " The Climate Risk Maturity Model is explained below:\n####\n{crmm_info}\n####")),
                chat.MessagesPlaceholder(variable_name='chat_history', optional=True),
                chat.HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['org_name'], template="Summarize how the organization, {org_name} is operating with respect to each of the six capabilities "
                     "defined in the Climate Risk Maturity Model. For each capability, provide a one sentence summary of their progress, and list the specific evidence that led to the conclusion. Research "
                     "the provider with respect to each capability if necessary.")),
                chat.MessagesPlaceholder(variable_name='agent_scratchpad')])

## Create the Agent

In [8]:
# Define the model (GPT-4)
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.25)

In [9]:
# Define agent
tools = [brave_tool, tavily_tool]
agent = create_openai_tools_agent(llm, tools, chat_prompt)

In [10]:
# Define agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


## Running the Agent

In [11]:
## Running the Agent
test_input_dict = {
    'crmm_info': crmm_raw,
    'org_name': 'Baltimore Gas and Electric'
}


In [12]:
agent_executor.invoke(test_input_dict)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Baltimore Gas and Electric executive level engagement and leadership in climate resilience'}`


[{'url': 'https://www.ethree.com/e3-releases-integrated-decarbonization-study-for-baltimore-gas-and-electric/', 'content': "NEWS: Energy and environmental policy E3 Releases Integrated Decarbonization Study for Baltimore Gas and Electric  NEWS: Energy and environmental policy E3 Releases Integrated Decarbonization Study for Baltimore Gas and Electric  the largest utility in Maryland, BGE has a critical role to play and is a key partner in decarbonizing the Maryland  The study finds that there are multiple paths to decarbonization and any future that meets Maryland’s net zero goalsE3 Releases Integrated Decarbonization Study for Baltimore Gas and Electric. October 7, 2022. The State of Maryland has begun its transition to a deeply decarbonized economy, building upon over a decade of climate change

{'crmm_info': 'Climate Resilience Maturity Model (CRMM) for Essential Services Providers  \t   \n\nExecutive Summary\n\nAll US cities are in some stage of awareness and action related to hardening their infrastructure assets and adapting their critical functions and processes to operate reliably in the more challenging conditions wrought by a changing climate. This model offers a mechanism for entities to baseline their present state and set goals for heightened resilience in the near-midterm future. When aggregated across the six sectors assessed, a city\'s resilience posture is made visible to all manner of interested stakeholders. Also please note:\n\n-\tCriteria and annual ratings are publicly accessible, though particularly sensitive data may be reserved for USG use.\n-\tThis model is fundamentally multi-sector and is designed to take cross-sector interdependencies into full account (e.g., energy, water, communications, transportation, etc.).\n-\tIntended to motivate accelerated a

### Baltimore Gas and Electric's Climate Risk Maturity Model Assessment

#### 1. Governance – Executive Level Engagement and Leadership
- **Summary:** Baltimore Gas and Electric (BGE) demonstrates a commitment to climate resilience through executive engagement, evidenced by leadership roles and strategic initiatives focused on climate change and energy policy.
- **Evidence:**
  - Carim V. Khouzami, President and CEO, oversees safety, reliability, customer service, and BGE's commitment to an inclusive culture addressing climate change ([source](https://www.exeloncorp.com/leadership-and-governance/executives/carim-v-khouzami)).
  - Leadership changes aimed at addressing climate change through accelerated electrification of BGE’s vehicle fleet and advocating for sound energy policy development ([source](https://www.bgenow.com/2021/11/19/bge-announces-senior-leadership-changes/)).

#### 2. Climate-Aware Planning
- **Summary:** BGE is actively involved in climate-aware planning, participating in studies and initiatives to decarbonize Maryland's economy and meet net-zero goals.
- **Evidence:**
  - Participation in an integrated decarbonization study for Maryland, exploring paths to achieve the state’s net-zero goals ([source](https://www.ethree.com/e3-releases-integrated-decarbonization-study-for-baltimore-gas-and-electric/)).

#### 3. Stakeholder and Community Collaboration
- **Summary:** BGE collaborates with stakeholders and communities on climate resilience, participating in partnerships and initiatives aimed at adapting to climate hazards and pursuing decarbonization goals.
- **Evidence:**
  - Participation in the Baltimore Social-Environmental Collaborative (BSEC), a partnership aimed at adapting to climate hazards and pursuing decarbonization goals ([source](https://hub.jhu.edu/2023/02/15/baltimore-social-environmental-collaborative/)).

#### 4. Resilience and Adaptation Actions
- **Summary:** BGE is taking steps towards resilience and adaptation by developing climate risk and adaptation plans, with an emphasis on equitable resiliency.
- **Evidence:**
  - Development of climate risk and adaptation plans, focusing on equitable resiliency and enhanced grid resilience for communities disproportionately impacted by climate change ([source](https://www.tdworld.com/transmission-reliability/article/21276745/bge-plans-for-equitable-resiliency)).

#### 5. Customer Engagement and Coordination
- **Summary:** BGE engages with customers and communities to address climate resilience, incorporating feedback into planning and prioritization processes.
- **Evidence:**
  - BGE's plan for net-zero carbon emissions by 2045 includes engaging communities and balancing resiliency with cost savings ([source](https://www.baltimoresun.com/2023/10/30/bge-ceo-planning-for-net-zero-carbon-emissions-in-2045-requires-balance-guest-commentary/)).

#### 6. Attention to Equity
- **Summary:** BGE prioritizes equitable resiliency in its climate resilience planning, focusing on communities disproportionately impacted by climate change.
- **Evidence:**
  - Emphasis on communities disproportionately impacted by climate change in the development of climate risk and adaptation plans ([source](https://www.tdworld.com/transmission-reliability/article/21276745/bge-plans-for-equitable-resiliency)).

Baltimore Gas and Electric is actively engaged in addressing climate resilience across multiple capabilities, demonstrating leadership, planning, collaboration, adaptation actions, customer .fforts.ach to climate resilience.